# Pulsar Star Prediction

<img width="1134" alt="methods" src="https://user-images.githubusercontent.com/28593767/112789439-0cda6480-9098-11eb-96e5-9fc335f3bfd8.png">

In [1]:
import numpy as np
import csv

np.random.seed(222)

RND_MEAN = 0
RND_STD = 0.003

LEARNING_RATE = 0.003

In [2]:
def binary_classification_exec(epoch_count=10,mb_size=10, report=1,train_rate=0.8,adjust_ratio = False): 
    binary_load_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report, train_rate)

Adjust_ration 라는 인자를 binary_load_dataset()로 할당하여 pulsar 데이터를 증폭시켜주는 역할을 수행한다.

In [20]:
# 오직 실수 데이터만 존재하므로 원-핫 벡터 과정은 생략 가능하다.
# 데이터 로드 및 처리 함수
def binary_load_dataset(adjust_ratio):
    pulsars, stars = [], []
    with open('data_nn/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        
        for row in csvreader:
            if row[8] == '1': pulsars.append(row)
            else:
                stars.append(row)
    global data, input_cnt, output_cnt 
    input_cnt, output_cnt = 8, 1
    star_cnt, pulsar_cnt = len(stars), len(pulsars)
    if adjust_ratio: 
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32') # Numpy 배열 구조로 변환
        for n in range(star_cnt):
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else: # adjust_ratio가 False의 경우 이전과 동일하게 실행된다
        data = np.zeros([star_cnt+pulsar_cnt,9]) 
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32') 
        data[star_cnt:,:] = np.asarray(pulsars, dtype='float32')

In [4]:
# 파라미터 초기화 함수
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [5]:
def train_and_test(epoch_count, mb_size, report, train_rate): 
    step_count = arrange_data(mb_size, train_rate)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count): 
        losses, accs = [], []
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n) 
            loss, acc = run_train(train_x, train_y) 
            losses.append(loss)
            accs.append(acc)
        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print("epoch{}:TRAIN - LOSS = {:5.3f},\n accuracy:{:5.3f}, precision:{:5.3f}, recall:{:5.3f}, f1:{:5.3f}".\
                  format(epoch+1,np.mean(losses), acc[0],acc[1],acc[2],acc[3]))
        
    final_acc = run_test(test_x, test_y)
    print("\n Final Test :\n accuracy:{:5.3f}, precision:{:5.3f}, recall:{:5.3f}, f1:{:5.3f}".\
            format(final_acc[0],final_acc[1],final_acc[2],final_acc[3]))

In [6]:
def arrange_data(mb_size, train_rate):
    global data, shuffle_map, test_begin_idx
    # 주의!!! Numpy의 메소드는 np.arange() 이다 (Arrange가 아니다!)
    shuffle_map = np.arange(data.shape[0]) 
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * train_rate) // mb_size
    test_begin_idx = step_count * mb_size
    
    return step_count

In [7]:
def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt 
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

In [8]:
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt 
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size * nth : mb_size * (nth + 1)]] 
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [9]:
def run_train(x, y):
    # 순전파 과정
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    # 역전파 과정
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp) 
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

In [10]:
def run_test(x, y):
    output, _ = forward_neuralnet(x) 
    accuracy = eval_accuracy(output, y) 
    return accuracy

In [11]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias 
    return output, x

In [12]:
def backprop_neuralnet(G_output, x): 
    global weight, bias
    g_output_w = x.transpose()
    G_w = np.matmul(g_output_w, G_output) 
    G_b = np.sum(G_output, axis=0)
    
    weight -= LEARNING_RATE * G_w 
    bias -= LEARNING_RATE * G_b

In [13]:
def forward_postproc(output, y):
    CEE = sigmoid_cross_entropy_with_logits(y,output)  # 시그모이드 교차 엔트로피 함수
    loss = np.mean(CEE)
    
    return loss, [y, output, CEE]

In [14]:
def backprop_postproc(G_loss, aux):
    y, output, CEE = aux
    G_loss = 1.0

    g_loss_entropy = 1.0 / np.prod(CEE.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y,output)

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy

    return G_output

In [15]:
def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

def relu(x):
    return np.maximum(x, 0)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

정확도, 정밀도, 재현율을 이용해 신경망을 평가한다.

In [16]:
def eval_accuracy(output,y): 
    est_yes = np.greater(output,0) 
    ans_yes = np.greater(y, 0.5)
    
    est_no = np.logical_not(est_yes) 
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes,ans_yes)) 
    fp = np.sum(np.logical_and(est_yes, ans_no)) 
    fn = np.sum(np.logical_and(est_no, ans_no)) 
    tn = np.sum(np.logical_and(est_no, ans_yes))
    
    accuracy = safe_div(tp+tn,tp+fp+fn+tn) 
    precision = safe_div(tp,tp+fp)
    recall = safe_div(tp,tp+fn)
    
    f1 = 2 * safe_div(recall*precision,recall+precision) 

    # 정확도, 정밀도, 재현율, F1 스코어를 리스트로 반환
    return [accuracy, precision, recall, f1] 

In [17]:
# 분모가 매우 작은 값일 때 0으로 변환하는 문제를 해결한다

def safe_div(p, q): 
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20:
        return np.sign(p) 
    return p / q

# 정확도 계산 함수 정의
결과값을 보면 학습 초기부터 정확도가 "너무" 높다.

그 이유는 데이터의 분포를 보면 알 수 있는데, 데이터의 타입이 0이 16259개, 1이 1639개로 일반 별의 분포가 90%를 차지하고 있다는 점이다.

따라서 정확도를 측정하는 과정에서 문제가 발생한다.

* 이러한 착시 현상으로 인해 정확도가 정확하게 보여지지 않는 상황을 타개하기 위해 신경망의 성능을 더욱 잘 보여줄 수 있는 또 다른 평가 지표가 필요하다.
* 대표적인 지표에는 **정밀도(precision)** 와 **재현율(recall)** 이 있다.
    + 정밀도는 신경망이 참으로 예측한 것 가운데 정답이 참인 비율을 의미한다. 
        - TP / (TP + FP)
    + 재현율은 거꾸로 정답이 참인 것들 가운데 신경망이 참으로 예측한 비율을 의미한다.
        - TP / (TP + FN)
    
> TP : 신경망의 추측이 '참(P)'이며, 데이터의 정답 또한 '참'(T)으로 정확하게 평가한 결과 (TRUE POSITIVE)
>
> TN : 신경망의 추측이 '거짓(N)'이며, 데이터의 정답은 '참'(T)으로 부정확하게 평가한 결과 (TRUE NEGATIVE)
>
> FP : 신경망의 추측이 '참(P)'이며, 데이터의 정답은 '거짓(F)'으로 부정확하게 평가한 결과 (FALSE POSITIVE)
>
> FN : 신경망의 추측이 '거짓(N)'이며, 데이터의 정답 또한 '거짓(F)'으로 정확하게 평가한 결과 (FALSE NEGATIVE)

정밀도와 재현율의 조화 평균인 **F1 score** 가 정확도 대신 사용되기도 한다.

조화 평균은 *역수의 차원에서 평균을 구하고 다시 역수*를 취해 원래 차원의 값으로 돌아오게 하여 구할 수 있다.

![f1](https://user-images.githubusercontent.com/28593767/112940203-ca805880-9167-11eb-9ad1-895a6674cb67.png)

In [18]:
binary_classification_exec(epoch_count=1000, report=100,mb_size = 10,adjust_ratio=False)

epoch100:TRAIN - LOSS = 0.345,
 accuracy:0.089, precision:0.864, recall:0.075, f1:0.138
epoch200:TRAIN - LOSS = 0.336,
 accuracy:0.089, precision:0.840, recall:0.080, f1:0.146
epoch300:TRAIN - LOSS = 0.297,
 accuracy:0.089, precision:0.898, recall:0.076, f1:0.140
epoch400:TRAIN - LOSS = 0.275,
 accuracy:0.089, precision:0.893, recall:0.074, f1:0.137
epoch500:TRAIN - LOSS = 0.282,
 accuracy:0.089, precision:0.944, recall:0.072, f1:0.134
epoch600:TRAIN - LOSS = 0.313,
 accuracy:0.089, precision:0.856, recall:0.078, f1:0.143
epoch700:TRAIN - LOSS = 0.314,
 accuracy:0.089, precision:0.825, recall:0.078, f1:0.143
epoch800:TRAIN - LOSS = 0.271,
 accuracy:0.089, precision:0.958, recall:0.071, f1:0.133
epoch900:TRAIN - LOSS = 0.290,
 accuracy:0.089, precision:0.829, recall:0.078, f1:0.142
epoch1000:TRAIN - LOSS = 0.295,
 accuracy:0.089, precision:0.986, recall:0.061, f1:0.115

 Final Test :
 accuracy:0.089, precision:0.986, recall:0.061, f1:0.115


In [21]:
binary_classification_exec(epoch_count=1000, report=100,mb_size = 10,adjust_ratio=True)

epoch100:TRAIN - LOSS = 0.985,
 accuracy:0.498, precision:0.590, recall:0.759, f1:0.664
epoch200:TRAIN - LOSS = 0.881,
 accuracy:0.498, precision:0.871, recall:0.520, f1:0.652
epoch300:TRAIN - LOSS = 0.929,
 accuracy:0.498, precision:0.797, recall:0.556, f1:0.655
epoch400:TRAIN - LOSS = 0.910,
 accuracy:0.498, precision:0.978, recall:0.475, f1:0.640
epoch500:TRAIN - LOSS = 0.870,
 accuracy:0.498, precision:0.968, recall:0.484, f1:0.645
epoch600:TRAIN - LOSS = 0.896,
 accuracy:0.498, precision:0.933, recall:0.498, f1:0.649
epoch700:TRAIN - LOSS = 0.902,
 accuracy:0.498, precision:0.691, recall:0.628, f1:0.658
epoch800:TRAIN - LOSS = 0.840,
 accuracy:0.498, precision:0.988, recall:0.464, f1:0.632
epoch900:TRAIN - LOSS = 0.868,
 accuracy:0.498, precision:0.994, recall:0.454, f1:0.623
epoch1000:TRAIN - LOSS = 0.868,
 accuracy:0.498, precision:0.980, recall:0.475, f1:0.640

 Final Test :
 accuracy:0.498, precision:0.980, recall:0.475, f1:0.640


 결과를 살펴보기 앞서 단순 정확도를 살펴보는 것보단 정밀도와 재현율 그리고 *F1 score*를 주의깊게 살펴봐야 한다. 

 최종 테스트 결과 pulsar 데이터를 증폭시킨 결과에 대한 모든 지표가 더 높은 결괏값을 출력했고 특히 재현율과 더불어 *F1 score*가 눈에 띄게 향상되었음을 알 수 있다. 
 
 하지만 이러한 데이터 증폭 방식을 무작정 적용하게 되면 이후 **과잉 적합(overfitting)** 문제가 발생할 수 있다.